In [1]:
suppressPackageStartupMessages(library('tidyverse'))
# suppressPackageStartupMessages(library('tidylog'))

# Mostra valores sem notação científica
options(scipen=999)

# library('sf')
# library('raster')

options(repr.matrix.max.cols=150) # repr.matrix.max.rows=100

In [2]:
rm_accent <- function(str,pattern="all") {
  if(!is.character(str))
    str <- as.character(str)
  pattern <- unique(pattern)
  if(any(pattern=="Ç"))
    pattern[pattern=="Ç"] <- "ç"
  symbols <- c(
    acute = "áéíóúÁÉÍÓÚýÝ",
    grave = "àèìòùÀÈÌÒÙ",
    circunflex = "âêîôûÂÊÎÔÛ",
    tilde = "ãõÃÕñÑ",
    umlaut = "äëïöüÄËÏÖÜÿ",
    cedil = "çÇ"
  )
  nudeSymbols <- c(
    acute = "aeiouAEIOUyY",
    grave = "aeiouAEIOU",
    circunflex = "aeiouAEIOU",
    tilde = "aoAOnN",
    umlaut = "aeiouAEIOUy",
    cedil = "cC"
  )
  accentTypes <- c("´","`","^","~","¨","ç")
  if(any(c("all","al","a","todos","t","to","tod","todo")%in%pattern)) # opcao retirar todos
    return(chartr(paste(symbols, collapse=""), paste(nudeSymbols, collapse=""), str))
  for(i in which(accentTypes%in%pattern))
    str <- chartr(symbols[i],nudeSymbols[i], str)
  return(str)
}

In [3]:
munis_list <- list(
  # Siglas de novas cidades além das originais do IPEA definidas com base em
  # https://informacoes.anatel.gov.br/legislacao/resolucoes/2005/403-resolucao-424
  munis_df = tribble(
    ~code_muni, ~abrev_muni, ~name_muni,        ~abrev_estado,  ~map_plot_ratio_wh,
    1302603,    "man",         "Manaus",                "AM",           1.27,
    2211001,    "tsa",         "Teresina",              "PI",           0.74, # 0.74 estimado
    2304400,    "for",         "Fortaleza",             "CE",           1.2,
    2408102,    "nat",         "Natal",                 "RN",           0.70,
    2507507,    "jpa",         "Joao Pessoa",           "PB",           0.74, # 0.74 estimado
    2611606,    "rec",         "Recife",                "PE",           0.68,
    2704302,    "mac",         "Maceio",                "AL",           0.74,
    2927408,    "sal",         "Salvador",              "BA",           1.36,
    3106200,    "bho",         "Belo Horizonte",        "MG",           0.69,
    3170206,    "ula",         "Uberlandia",            "MG",           0.74, # 0.74 estimado
    3205309,    "vta",         "Vitoria",               "ES",           0.74, # 0.74 estimado
    3304557,    "rio",         "Rio de Janeiro",        "RJ",           1.91,
    3509502,    "cam",         "Campinas",              "SP",           1.20,
    3534401,    "oco",         "Osasco",                "SP",           0.74, # 0.74 estimado
    3547809,    "sne",         "Santo Andre",           "SP",           0.74, # 0.74 estimado
    3549904,    "sjc",         "Sao Jose dos Campos",   "SP",           0.74, # 0.74 estimado
    3550308,    "spo",         "Sao Paulo",             "SP",           0.65,
    4106902,    "cur",         "Curitiba",              "PR",           0.62,
    4113700,    "lda",         "Londrina",              "PR",           0.74, # 0.74 estimado
    5002704,    "cgr",         "Campo Grande",          "MS",           0.87,
    5208707,    "goi",         "Goiania",               "GO",           0.93
#    1501402,    "bel",       "Belem",           "PA",           0.65,
#    2111300,    "slz",       "Sao Luis",        "MA",           0.78,
#    3301702,    "duq",       "Duque de Caxias", "RJ",           0.61,
#    3304904,    "sgo",       "Sao Goncalo",     "RJ",           1.21,
#    3518800,    "gua",       "Guarulhos",       "SP",           0.91,
#    4314902,    "poa",       "Porto Alegre",    "RS",           0.75,
#    5300108,    "bsb",       "Brasilia",        "DF",           1.71
  ) )

munis <- 
  as.data.frame(munis_list$munis_df) %>% 
  mutate(IBGE = str_sub(code_muni, start = 0, end = 6)) %>% 
  dplyr::select(IBGE, name_muni)

head(munis, 3)

,IBGE,name_muni
,<chr>,<chr>
1,130260,Manaus
2,221100,Teresina
3,230440,Fortaleza


# 06_cnes_saude

In [4]:
open_file <- '../../../indice-mobilidade_dados/06_cnes_saude/2019/saude_2019_filter_geocode_revisto_gmaps.rds'
cnes_saude <- read_rds(open_file)

# Filtrar somente linhas onde o georreferenciamento foi refeito
cnes_ok <- cnes_saude %>% filter(rev_latlong == 0)
cnes_rev <- cnes_saude %>% filter(rev_latlong == 1)
dim(cnes_ok)
dim(cnes_rev)

dim(cnes_saude)
head(cnes_saude, 2)

[1] 5300   91

[1] 1461   91

[1] 6761   91

CO_UNIDADE,IBGE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO,CO_TIPO_UNIDADE,DS_TIPO_UNIDADE,CO_GESTAO_00,CO_GESTAO_01,CO_GESTAO_02,CO_GESTAO_03,CO_GESTAO_04,CO_GESTAO_05,CO_GESTAO_06,TP_PROG_0,TP_PROG_1,TP_PROG_2,CO_CONVENIO_01,CO_CONVENIO_02,CO_CONVENIO_03,CO_CONVENIO_04,CO_CONVENIO_05,CO_CONVENIO_06,CO_CONVENIO_07,CO_ATENDIMENTO_PRESTADO_01,CO_ATENDIMENTO_PRESTADO_02,CO_ATENDIMENTO_PRESTADO_03,CO_ATENDIMENTO_PRESTADO_04,CO_ATENDIMENTO_PRESTADO_05,CO_ATENDIMENTO_PRESTADO_06,CO_ATENDIMENTO_PRESTADO_07,health_low,health_med,health_high,latitude,longitude,latlong_id,rev_latlong,end_geocode,gmaps_type,gmaps_loctype,gmaps_address
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
2611607810032,261160,7810032,NA,3,1,FARIAS ANALISES CLINICAS EIRELI,FARIAS ANALISES CLINICAS,AVENIDA NORTE MIGUEL ARRAES DE ALENCAR,2535,NA,ENCRUZILHADA,52041080,1,1,02,NA,NA,NA,erikacfarias@laboaratorio.ciac.com.br,NA,22235187000145,04,03,0705340117,18-fev-2019 00:00:00,2,18-fev-2020 00:00:00,1,36,03,26,261160,30/12/2019,FLAVIA,07663846400,0140,NA,NA,NA,-8.041001,-34.88614,30/12/2019,FLAVIA,2305,N,NA,S,NA,M,04/09/2015,018,002,S,36,CLINICA/CENTRO DE ESPECIALIDADE,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,-8.0410008,-34.886136,-8.0410008-34.886136,0,NA,NA,NA,NA
1302607695667,130260,7695667,NA,3,1,I S DE SOUZA,AMAZON CLINICAL CENTER,R BERNARDO MICHILES,1296,SALA 5 2 ANDAR,PETROPOLIS,69067000,001,NA,NA,NA,92 33467384,NA,amazonclinical@gmail.com,NA,13696457000124,04,NA,NA,NA,NA,NA,NA,60,NA,13,130260,27/10/2017,AMAZON,43942040204,AM-011527-0,NA,04,NA,NA,NA,NA,NA,2062,S,NA,S,NA,E,13/04/2015,000,022,NA,NA,NA,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NA,NA,NANA,0,NA,NA,NA,NA


In [5]:
nrow(cnes_ok)+nrow(cnes_rev) == nrow(cnes_saude)

[1] TRUE

In [6]:
# # cnes_check são os endereços georreferenciados via gmaps com tratamento 
# # de colunas para checagem da qualidade do geocode
# cnes_check <- 
#     cnes_rev %>%
#     # Adicionar info de municipios
#     left_join(munis, by = 'IBGE') %>% 
# #     select(IBGE, latlong_id, NO_LOGRADOURO, NU_ENDERECO, name_muni, CO_CEP, gmaps_address) %>%     
#     mutate(
#         # Criar coluna de CEP do endereço encontrado pelo Google
#         gmaps_CEP = str_replace(gmaps_address, '(.+, ){2,}(\\d{5})(-)?(\\d{3})?(,.+)', '\\2\\4'),
#         # Criar coluna do logradouro
#         gmaps_log = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\1'),
#         gmaps_log = toupper(rm_accent(gmaps_log)),
#         gmaps_log = str_replace(gmaps_log, 'R\\.', 'RUA'),
#         gmaps_log = str_replace(gmaps_log, 'AV\\.', 'AV'),
#         gmaps_log = str_replace(gmaps_log, 'AV ', 'AVENIDA '),
#         # Ajeitar coluna de NO_LOGRADOURO
#         logradouro_rev = str_replace(NO_LOGRADOURO, 'AV\\.', 'AV'),
#         logradouro_rev = str_replace(NO_LOGRADOURO, 'AV ', 'AVENIDA '),
#         # Criar coluna do numeral
#         gmaps_num = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\2'),
#         # Criar coluna da cidade
#         gmaps_cid = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\4'),
#         # Ajeitar coluna de cidade para se assemelhar à name_muni
#         gmaps_cid = toupper(rm_accent(gmaps_cid)),
#         # Ajeitar coluna name_muni
#         name_muni = toupper(name_muni))
          

# dim(cnes_check)
# head(cnes_check)

In [7]:
# cnes_check são os endereços georreferenciados via gmaps com tratamento 
# de colunas para checagem da qualidade do geocode
cnes_check <- 
    cnes_rev %>%
    # Adicionar info de municipios
    left_join(munis, by = 'IBGE') %>% 
#     select(IBGE, latlong_id, NO_LOGRADOURO, NU_ENDERECO, name_muni, CO_CEP, gmaps_address) %>%     
    mutate(
        # Criar coluna de CEP do endereço encontrado pelo Google
        gmaps_CEP = str_replace(gmaps_address, '(.+, ){2,}(\\d{5})(-)?(\\d{3})?(,.+)', '\\2\\4'),
        # Criar coluna do logradouro
        gmaps_log = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\1'),
        gmaps_log = toupper(rm_accent(gmaps_log)),
        gmaps_log = str_replace(gmaps_log, '^AV\\.', 'AV'),
        gmaps_log = str_replace(gmaps_log, '^AV ', 'AVENIDA '),        
        gmaps_log = str_replace(gmaps_log, '^R\\.', 'RUA'),        
        gmaps_log = str_replace(gmaps_log, '^TV\\.', 'TRAVESSA'),        
        gmaps_log = str_replace(gmaps_log, 'DR\\.', 'DR'),
        gmaps_log = str_replace(gmaps_log, 'MAL\\.', 'MARECHAL'),
        gmaps_log = str_replace(gmaps_log, 'SR\\.', 'SENHOR'),
        # Ajeitar coluna de NO_LOGRADOURO
        logradouro_rev = str_replace(NO_LOGRADOURO, '^AV\\.', 'AV'),
        logradouro_rev = str_replace(NO_LOGRADOURO, '^AV ', 'AVENIDA '),
        # Criar coluna do numeral
        gmaps_num = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\2'),
        # Criar coluna da cidade
        gmaps_cid = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\4'),
        # Ajeitar coluna de cidade para se assemelhar à name_muni
        gmaps_cid = toupper(rm_accent(gmaps_cid)),
        # Ajeitar coluna name_muni
        name_muni = toupper(name_muni),
        
        # Para endereços sem número, precisamos de uma nova regex para isolar o logradouro...        
        gmaps_log2 = str_replace(gmaps_log, '(.+){1} - (.+)', '\\1'),
        # ... para isolar a cidade...
        gmaps_cid2 = str_replace(gmaps_log2, '(.+){1}, (.+)', '\\2'),
        # ... e finalmente ajeitar o logradouro
        gmaps_log2 = str_replace(gmaps_log2, '(.+){1}, (.+)', '\\1'),
        gmaps_log2 = str_replace(gmaps_log2, '(.+){1} - (.+)', '\\1'),
        )
          

dim(cnes_check)
head(cnes_check)

[1] 1461   99

CO_UNIDADE,IBGE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO,CO_TIPO_UNIDADE,DS_TIPO_UNIDADE,CO_GESTAO_00,CO_GESTAO_01,CO_GESTAO_02,CO_GESTAO_03,CO_GESTAO_04,CO_GESTAO_05,CO_GESTAO_06,TP_PROG_0,TP_PROG_1,TP_PROG_2,CO_CONVENIO_01,CO_CONVENIO_02,CO_CONVENIO_03,CO_CONVENIO_04,CO_CONVENIO_05,CO_CONVENIO_06,CO_CONVENIO_07,CO_ATENDIMENTO_PRESTADO_01,CO_ATENDIMENTO_PRESTADO_02,CO_ATENDIMENTO_PRESTADO_03,CO_ATENDIMENTO_PRESTADO_04,CO_ATENDIMENTO_PRESTADO_05,CO_ATENDIMENTO_PRESTADO_06,CO_ATENDIMENTO_PRESTADO_07,health_low,health_med,health_high,latitude,longitude,latlong_id,rev_latlong,end_geocode,gmaps_type,gmaps_loctype,gmaps_address,name_muni,gmaps_CEP,gmaps_log,logradouro_rev,gmaps_num,gmaps_cid,gmaps_log2,gmaps_cid2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3550309414843,355030,9414843,NA,3,1,ASSOCIACAO BENEFICENTE EBENEZER,HORA CERTA MOVEL CAPELA DO SOCORRO CIES,RUA CASSIANO DOS SANTOS,43,NA,IPIRANGA,04827110,NA,NA,NA,NA,NA,NA,NA,NA,06950310000153,04,03,NA,NA,NA,NA,NA,40,03,35,355030,29/11/2019,CADASTRO,14241029876,84040,NA,10,NA,-23.54800,-46.63600,22/08/2019,CADASTRO,3999,N,NA,S,NA,M,04/02/2018,016,001,S,NA,NA,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,-23.7228651,-46.6985602,-23.548-46.636,1,"RUA CASSIANO DOS SANTOS, 43, IPIRANGA, Sao Paulo, 04827110",premise,rooftop,"r. cassiano dos santos, 43 - rio bonito, são paulo - sp, 04827-110, brazil",SAO PAULO,04827110,RUA CASSIANO DOS SANTOS,RUA CASSIANO DOS SANTOS,43,SAO PAULO,RUA CASSIANO DOS SANTOS,RUA CASSIANO DOS SANTOS
2927407872828,292740,7872828,13927801000572,3,3,SECRETARIA MUNICIPAL DE SAUDE DE SALVADOR,USF NOVA BRASILIA,AV ALIOMAR BALEEIRO KM 7 5,S/N,EST VELHA AEROPORTO,NOVA BRASILIA,41350275,001,NA,0010,NA,(71) 36116834/7834,NA,usfnovabrasiliaadm@gmail.com,NA,NA,04,03,NA,NA,NA,NA,NA,02,03,29,292740,20/01/2020,JAMILE,65591313434,NA,NA,NA,NA,-12.97100,-38.51100,21/08/2019,MDORA,1244,N,NA,S,NA,M,05/12/2015,001,012,NA,NA,NA,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,NA,NA,-12.971-38.511,1,"AV ALIOMAR BALEEIRO KM 7 5, NOVA BRASILIA, Salvador, 41350275",NA,NA,NA,SALVADOR,NA,NA,AVENIDA ALIOMAR BALEEIRO KM 7 5,NA,NA,NA,NA
4113707754280,411370,7754280,75771477000170,3,3,PREFEITURA DO MUNICIPIO DE LONDRINA,RESGATE AEREO GRAER,RUA DIB LIBOS,80,NA,CENTRO,86010470,17,NA,NA,NA,NA,NA,NA,NA,NA,04,02,NA,NA,NA,NA,NA,42,06,41,411370,01/07/2019,FCES,93523971934,124766,NA,NA,NA,-23.31441,-51.14937,17/01/2018,FCES,1244,S,NA,N,NA,M,25/06/2015,008,007,NA,NA,NA,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-23.3140829,-51.1495346,-23.314405-51.149372,1,"RUA DIB LIBOS, 80, CENTRO, Londrina, 86010470",premise,rooftop,"r. dib libos, 80 

#### Fazer marcações

In [8]:
# Marcação 1: Onde gmaps_address for NA, marcar como rever
cnes_check <- cnes_check %>% mutate(rever_geocode = case_when(is.na(gmaps_address) ~ 'sim', TRUE ~ 'não'))

# Marcação 2: Onde CEPs forem diferentes (original x geocode), marcar como rever
cnes_check <- cnes_check %>% mutate(rever_geocode = case_when(gmaps_CEP != CO_CEP ~ 'sim', 
                                                              TRUE ~ rever_geocode))

# Marcação 3: Mesmo que os CEPs sejam diferentes, se o endereço, número 
# e cidade forem iguais, marcar que não precisa rever
cnes_check <- 
    cnes_check %>% 
    mutate(rever_geocode = case_when(logradouro_rev == gmaps_log &
                                     NU_ENDERECO == gmaps_num &
                                     name_muni == gmaps_cid
                                     ~ 'não',
                                     TRUE ~ rever_geocode))

# Marcação 4: Para endereços que já não possuíam numeral, se 
# endereço e cidade forem iguais, marcar que não precisa rever
cnes_check <- 
    cnes_check %>% 
    mutate(rever_geocode = case_when(logradouro_rev == gmaps_log2 &
                                     NU_ENDERECO == 'S/N' &
                                     name_muni == gmaps_cid2
                                     ~ 'não',
                                     TRUE ~ rever_geocode))

head(cnes_check)

CO_UNIDADE,IBGE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO,CO_TIPO_UNIDADE,DS_TIPO_UNIDADE,CO_GESTAO_00,CO_GESTAO_01,CO_GESTAO_02,CO_GESTAO_03,CO_GESTAO_04,CO_GESTAO_05,CO_GESTAO_06,TP_PROG_0,TP_PROG_1,TP_PROG_2,CO_CONVENIO_01,CO_CONVENIO_02,CO_CONVENIO_03,CO_CONVENIO_04,CO_CONVENIO_05,CO_CONVENIO_06,CO_CONVENIO_07,CO_ATENDIMENTO_PRESTADO_01,CO_ATENDIMENTO_PRESTADO_02,CO_ATENDIMENTO_PRESTADO_03,CO_ATENDIMENTO_PRESTADO_04,CO_ATENDIMENTO_PRESTADO_05,CO_ATENDIMENTO_PRESTADO_06,CO_ATENDIMENTO_PRESTADO_07,health_low,health_med,health_high,latitude,longitude,latlong_id,rev_latlong,end_geocode,gmaps_type,gmaps_loctype,gmaps_address,name_muni,gmaps_CEP,gmaps_log,logradouro_rev,gmaps_num,gmaps_cid,gmaps_log2,gmaps_cid2,rever_geocode
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3550309414843,355030,9414843,NA,3,1,ASSOCIACAO BENEFICENTE EBENEZER,HORA CERTA MOVEL CAPELA DO SOCORRO CIES,RUA CASSIANO DOS SANTOS,43,NA,IPIRANGA,04827110,NA,NA,NA,NA,NA,NA,NA,NA,06950310000153,04,03,NA,NA,NA,NA,NA,40,03,35,355030,29/11/2019,CADASTRO,14241029876,84040,NA,10,NA,-23.54800,-46.63600,22/08/2019,CADASTRO,3999,N,NA,S,NA,M,04/02/2018,016,001,S,NA,NA,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,-23.7228651,-46.6985602,-23.548-46.636,1,"RUA CASSIANO DOS SANTOS, 43, IPIRANGA, Sao Paulo, 04827110",premise,rooftop,"r. cassiano dos santos, 43 - rio bonito, são paulo - sp, 04827-110, brazil",SAO PAULO,04827110,RUA CASSIANO DOS SANTOS,RUA CASSIANO DOS SANTOS,43,SAO PAULO,RUA CASSIANO DOS SANTOS,RUA CASSIANO DOS SANTOS,não
2927407872828,292740,7872828,13927801000572,3,3,SECRETARIA MUNICIPAL DE SAUDE DE SALVADOR,USF NOVA BRASILIA,AV ALIOMAR BALEEIRO KM 7 5,S/N,EST VELHA AEROPORTO,NOVA BRASILIA,41350275,001,NA,0010,NA,(71) 36116834/7834,NA,usfnovabrasiliaadm@gmail.com,NA,NA,04,03,NA,NA,NA,NA,NA,02,03,29,292740,20/01/2020,JAMILE,65591313434,NA,NA,NA,NA,-12.97100,-38.51100,21/08/2019,MDORA,1244,N,NA,S,NA,M,05/12/2015,001,012,NA,NA,NA,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,NA,NA,-12.971-38.511,1,"AV ALIOMAR BALEEIRO KM 7 5, NOVA BRASILIA, Salvador, 41350275",NA,NA,NA,SALVADOR,NA,NA,AVENIDA ALIOMAR BALEEIRO KM 7 5,NA,NA,NA,NA,sim
4113707754280,411370,7754280,75771477000170,3,3,PREFEITURA DO MUNICIPIO DE LONDRINA,RESGATE AEREO GRAER,RUA DIB LIBOS,80,NA,CENTRO,86010470,17,NA,NA,NA,NA,NA,NA,NA,NA,04,02,NA,NA,NA,NA,NA,42,06,41,411370,01/07/2019,FCES,93523971934,124766,NA,NA,NA,-23.31441,-51.14937,17/01/2018,FCES,1244,S,NA,N,NA,M,25/06/2015,008,007,NA,NA,NA,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-23.3140829,-51.1495346,-23.314405-51.149372,1,"RUA DIB LIBOS, 80, CENTRO, Londrina, 86010470",premis

In [9]:
# # cnes_check são os endereços georreferenciados via gmaps com tratamento 
# # de colunas para checagem da qualidade do geocode
# cnes_check <- 
#     cnes_rev %>%
#     # Adicionar info de municipios
#     left_join(munis, by = 'IBGE') %>% 
#     select(IBGE, NO_LOGRADOURO, NU_ENDERECO, name_muni, CO_CEP, gmaps_address) %>%     
#     mutate(
#         # Criar coluna de CEP do endereço encontrado pelo Google
#         gmaps_CEP = str_replace(gmaps_address, '(.+, ){2,}(\\d{5})(-)?(\\d{3})?(,.+)', '\\2\\4'),
#         # Criar coluna do logradouro
#         gmaps_log = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\1'),
#         gmaps_log = toupper(rm_accent(gmaps_log)),
#         gmaps_log = str_replace(gmaps_log, '^AV\\.', 'AV'),
#         gmaps_log = str_replace(gmaps_log, '^AV ', 'AVENIDA '),        
#         gmaps_log = str_replace(gmaps_log, '^R\\.', 'RUA'),        
#         gmaps_log = str_replace(gmaps_log, '^TV\\.', 'TRAVESSA'),        
#         gmaps_log = str_replace(gmaps_log, 'DR\\.', 'DR'),
#         gmaps_log = str_replace(gmaps_log, 'SR\\.', 'SENHOR'),
#         # Ajeitar coluna de NO_LOGRADOURO
#         logradouro_rev = str_replace(NO_LOGRADOURO, '^AV\\.', 'AV'),
#         logradouro_rev = str_replace(NO_LOGRADOURO, '^AV ', 'AVENIDA '),
#         # Criar coluna do numeral
#         gmaps_num = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\2'),
#         # Criar coluna da cidade
#         gmaps_cid = str_replace(gmaps_address, '(.+){1}, (\\d+)? - (.+, ){1}(.+){1} - (.+, ){1}(\\d{5})(-)?(\\d{3})?(,.+)', '\\4'),
#         # Ajeitar coluna de cidade para se assemelhar à name_muni
#         gmaps_cid = toupper(rm_accent(gmaps_cid)),
#         # Ajeitar coluna name_muni
#         name_muni = toupper(name_muni),
        
#         # Para endereços sem número, precisamos de uma nova regex para isolar o logradouro...        
#         gmaps_log2 = str_replace(gmaps_log, '(.+){1} - (.+)', '\\1'),
#         # ... para isolar a cidade...
#         gmaps_cid2 = str_replace(gmaps_log2, '(.+){1}, (.+)', '\\2'),
#         # ... e finalmente ajeitar o logradouro
#         gmaps_log2 = str_replace(gmaps_log2, '(.+){1}, (.+)', '\\1'),
#         gmaps_log2 = str_replace(gmaps_log2, '(.+){1} - (.+)', '\\1'),
# )

# # Marcação 1: Onde gmaps_address for NA, marcar como rever
# cnes_check <- cnes_check %>% mutate(rever_geocode = case_when(is.na(gmaps_address) ~ 'sim', TRUE ~ 'não'))

# # Marcação 2: Onde CEPs forem diferentes (original x geocode), marcar como rever
# cnes_check <- cnes_check %>% mutate(rever_geocode = case_when(gmaps_CEP != CO_CEP ~ 'sim', 
#                                                               TRUE ~ rever_geocode))

# # Marcação 3: Mesmo que os CEPs sejam diferentes, se o endereço, número 
# # e cidade forem iguais, marcar que não precisa rever
# cnes_check <- 
#     cnes_check %>% 
#     mutate(rever_geocode = case_when(logradouro_rev == gmaps_log &
#                                      NU_ENDERECO == gmaps_num &
#                                      name_muni == gmaps_cid
#                                      ~ 'não',
#                                      TRUE ~ rever_geocode))

# # Marcação 4: Para endereços que já não possuíam númeral, se 
# # endereço e cidade forem iguais, marcar que não precisa rever
# cnes_check <- 
#     cnes_check %>% 
#     mutate(rever_geocode = case_when(logradouro_rev == gmaps_log2 &
#                                      NU_ENDERECO == 'S/N' &
#                                      name_muni == gmaps_cid2
#                                      ~ 'não',
#                                      TRUE ~ rever_geocode))

# cnes_check <- cnes_check %>% filter(IBGE == '240810' & rever_geocode == 'sim' & NU_ENDERECO == 'S/N') %>% 
#     select(NO_LOGRADOURO, gmaps_log, gmaps_log2, gmaps_cid2)

# dim(cnes_check)
# head(cnes_check, 20)

In [10]:
cnes_check %>% group_by(rever_geocode) %>% tally()

rever_geocode,n
<chr>,<int>
não,1142
sim,319


In [11]:
# Em Natal
cnes_check %>% filter(IBGE == '240810') %>% group_by(rever_geocode) %>% tally()

rever_geocode,n
<chr>,<int>
não,58
sim,18


In [12]:
cnes_check %>% 
    filter(IBGE == '240810' & rever_geocode == 'sim') %>%
    select(end_geocode, gmaps_address)

end_geocode,gmaps_address
<chr>,<chr>
"AV DEODORO, 498, CIDADE ALTA, Natal, 59025600","av. deodoro da fonseca, 498 - petrópolis, natal - rn, 59012-240, brazil"
"RUA XAVIER DA SILVEIRA, 05, NOVA DESCOBERTA, Natal, 59075000","av. xavier da silveira, 5 - tirol, natal - rn, 59056-700, brazil"
"AV ALEXANDRINO DE ALENCAR, 1900, TIROL, Natal, 59015350","av. alm. alexandrino de alencar, 1900 - tirol, natal - rn, 59030-660, brazil"
"AVENIDA ALEXANDRINO DE ALENCAR, 908, LAGOA SECA, Natal, 59030350","av. alm. alexandrino de alencar, 908 - barro vermelho, natal - rn, 59022-350, brazil"
"MIGUEL CASTRO, 1141, LAGOA NOVA, Natal, 59075740","av. miguel castro, 1141 - lagoa nova, natal - rn, 59052-700, brazil"
"RUA SAO TIAGO, IGAPO, Natal, 59104040","rua fefita santiago - igapó, natal - rn, 59104-056, brazil"
"AV DR JOAO MEDEIROS, POTENGI, Natal, 59108200","av. dr. joão medeiros filho - potengi, natal - rn, 59104-200, brazil"
"DESPORTISTA JEREMIAS PINHEIRO DA CAMARA FILHO, 2, PONTA NEGRA, Natal, 59091250","r. desportista jeremias pinheiro da câmara filho, 2 - neópolis, natal - rn, 59088-105, brazil"
"RUA JOAQUIM MANOEL, 720, PETROPOLIS, Natal, 59012330","rua coronel joaquim manoel, 720 - petrópolis, natal - rn, 59014-077, brazil"


In [13]:
# Retirar colunas temporárias criadas para merge final
cnes_check <- 
    cnes_check %>% 
    select(-c(name_muni, logradouro_rev, gmaps_CEP, gmaps_log, gmaps_num, gmaps_cid, gmaps_log2, gmaps_cid2))

dim(cnes_check)

[1] 1461   92

In [14]:
dim(cnes_saude)

[1] 6761   91

In [15]:
this <- names(cnes_saude)
that <- names(cnes_check)
setdiff(this, that)
setdiff(that, this)

character(0)

[1] "rever_geocode"

In [16]:
# Juntar dataframes de georreferenciamento ok e dos que talvez
# precisem de revisão para exportar
cnes_out <- cnes_ok %>% mutate(rever_geocode = NA) %>% rbind(cnes_check)

dim(cnes_out)
head(cnes_out, 3)

[1] 6761   92

CO_UNIDADE,IBGE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO,CO_TIPO_UNIDADE,DS_TIPO_UNIDADE,CO_GESTAO_00,CO_GESTAO_01,CO_GESTAO_02,CO_GESTAO_03,CO_GESTAO_04,CO_GESTAO_05,CO_GESTAO_06,TP_PROG_0,TP_PROG_1,TP_PROG_2,CO_CONVENIO_01,CO_CONVENIO_02,CO_CONVENIO_03,CO_CONVENIO_04,CO_CONVENIO_05,CO_CONVENIO_06,CO_CONVENIO_07,CO_ATENDIMENTO_PRESTADO_01,CO_ATENDIMENTO_PRESTADO_02,CO_ATENDIMENTO_PRESTADO_03,CO_ATENDIMENTO_PRESTADO_04,CO_ATENDIMENTO_PRESTADO_05,CO_ATENDIMENTO_PRESTADO_06,CO_ATENDIMENTO_PRESTADO_07,health_low,health_med,health_high,latitude,longitude,latlong_id,rev_latlong,end_geocode,gmaps_type,gmaps_loctype,gmaps_address,rever_geocode
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
2611607810032,261160,7810032,NA,3,1,FARIAS ANALISES CLINICAS EIRELI,FARIAS ANALISES CLINICAS,AVENIDA NORTE MIGUEL ARRAES DE ALENCAR,2535,NA,ENCRUZILHADA,52041080,1,1,02,NA,NA,NA,erikacfarias@laboaratorio.ciac.com.br,NA,22235187000145,04,03,0705340117,18-fev-2019 00:00:00,2,18-fev-2020 00:00:00,1,36,03,26,261160,30/12/2019,FLAVIA,07663846400,0140,NA,NA,NA,-8.041001,-34.88614,30/12/2019,FLAVIA,2305,N,NA,S,NA,M,04/09/2015,018,002,S,36,CLINICA/CENTRO DE ESPECIALIDADE,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,-8.0410008,-34.886136,-8.0410008-34.886136,0,NA,NA,NA,NA,NA
1302607695667,130260,7695667,NA,3,1,I S DE SOUZA,AMAZON CLINICAL CENTER,R BERNARDO MICHILES,1296,SALA 5 2 ANDAR,PETROPOLIS,69067000,001,NA,NA,NA,92 33467384,NA,amazonclinical@gmail.com,NA,13696457000124,04,NA,NA,NA,NA,NA,NA,60,NA,13,130260,27/10/2017,AMAZON,43942040204,AM-011527-0,NA,04,NA,NA,NA,NA,NA,2062,S,NA,S,NA,E,13/04/2015,000,022,NA,NA,NA,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NA,NA,NANA,0,NA,NA,NA,NA,NA
2304409526293,230440,9526293,04885197000144,3,3,SECRETARIA MUNICIPAL DA SAUDE,UPA UNIDADE DE PRONTO ATENDIMENTO EDSON QUEIROZ,AVENIDA CONTORNO,S/N,ATRAS DA UNIFOR,EDSON QUEIROZ,60540272,NA,NA,0006,NA,35232600,NA,patriciatbraun@gmail.com,NA,NA,04,01,NA,NA,NA,NA,NA,73,06,23,230440,29/11/2019,MILENA,01652395300,NA,NA,NA,NA,-3.768284,-38.47335,12/08/2019,FGALDINO,1031,S,NA,S,NA,M,17/06/2018,008,007,NA,73,PRONTO ATENDIMENTO,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-3.768284,-38.473346,-3.768284-38.473346,0,NA,NA,NA,NA,NA


In [17]:
write_delim(cnes_out, 'CNES_checagem_visual_georrefs.csv', delim = ';')